In [ ]:
%reset -f

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import ee
ee.Authenticate()


Mounted at /content/drive


In [ ]:
#Future Windows Accumulation
import ee

ee.Initialize(project='ee-andrewfullhart')

maca_ic = ee.ImageCollection('NASA/NEX-DCP30')

order_months = ee.List([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

year_windows = ee.List([[1974, 2013], [2000, 2029], [2010, 2039], [2020, 2049], [2030, 2059],
                        [2040, 2069], [2050, 2079], [2060, 2089], [2070, 2099]])

area_shp = ee.Geometry.Rectangle([-121, 30, -102, 43], 'EPSG:4326', False)

def year_fn(yrwinobj):
  start_year = ee.Number(ee.List(yrwinobj).get(0))
  end_year = ee.Number(ee.List(yrwinobj).get(1))
  start = ee.Date.fromYMD(ee.Number(start_year), 1, 1)
  end = ee.Date.fromYMD(ee.Number(end_year).add(1), 1, 1)
  year_ic = maca_ic.filterDate(start, end).select('pr')
  modelfilter = ee.Filter.Or(
                ee.Filter.eq('scenario', 'historical'),
                ee.Filter.eq('scenario', 'rcp45'))
  year_ic = year_ic.filter(modelfilter)
  year_ic = year_ic.filter(ee.Filter.eq('model', 'CCSM4'))

  def month_fn(mobj):
    m = ee.Number(mobj)
    month_ic = year_ic.filter(ee.Filter.calendarRange(m, m, 'month'))
    mean_im = month_ic.reduce(ee.Reducer.mean()).clip(area_shp).multiply(86400)
    return mean_im

  mean_ic_list = order_months.map(month_fn)
  return mean_ic_list

nested_mean_ic_list = year_windows.map(year_fn)
mean_ic_list = nested_mean_ic_list.flatten()


yrwin_i = 0
mo_i = 0
for i in range(12*year_windows.size().getInfo()):
  mo_i += 1
  start_year = ee.Number(ee.List(year_windows.get(yrwin_i)).get(0)).getInfo()
  end_year = ee.Number(ee.List(year_windows.get(yrwin_i)).get(1)).getInfo()

  eimage = ee.Image(mean_ic_list.get(i))

  task = ee.batch.Export.image.toDrive(
          image=eimage,
          description='accm_{}_{}_{}'.format(str(start_year), str(end_year), str(mo_i)),
          folder='GEE_Downloads',
          region=area_shp,
          scale=100,
          crs='EPSG:4326',
          maxPixels=1e13)

  task.start()

  if mo_i == 12:
    mo_i = 0
    yrwin_i += 1




In [ ]:
#Future Windows Max Temperature
import ee

ee.Initialize(project='ee-andrewfullhart')

maca_ic = ee.ImageCollection('NASA/NEX-DCP30')

order_months = ee.List([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

year_windows = ee.List([[1974, 2013], [2000, 2029], [2010, 2039], [2020, 2049], [2030, 2059],
                        [2040, 2069], [2050, 2079], [2060, 2089], [2070, 2099]])

area_shp = ee.Geometry.Rectangle([-121, 30, -102, 43], 'EPSG:4326', False)

def year_fn(yrwinobj):
  start_year = ee.Number(ee.List(yrwinobj).get(0))
  end_year = ee.Number(ee.List(yrwinobj).get(1))
  start = ee.Date.fromYMD(ee.Number(start_year), 1, 1)
  end = ee.Date.fromYMD(ee.Number(end_year).add(1), 1, 1)
  year_ic = maca_ic.filterDate(start, end).select('tasmax')
  modelfilter = ee.Filter.Or(
                ee.Filter.eq('scenario', 'historical'),
                ee.Filter.eq('scenario', 'rcp45'))
  year_ic = year_ic.filter(modelfilter)
  year_ic = year_ic.filter(ee.Filter.eq('model', 'CCSM4'))

  def month_fn(mobj):
    m = ee.Number(mobj)
    month_ic = year_ic.filter(ee.Filter.calendarRange(m, m, 'month'))
    mean_im = month_ic.reduce(ee.Reducer.mean()).clip(area_shp).subtract(273.15)
    return mean_im

  mean_ic_list = order_months.map(month_fn)
  return mean_ic_list

nested_mean_ic_list = year_windows.map(year_fn)
mean_ic_list = nested_mean_ic_list.flatten()


yrwin_i = 0
mo_i = 0
for i in range(12*year_windows.size().getInfo()):
  mo_i += 1
  start_year = ee.Number(ee.List(year_windows.get(yrwin_i)).get(0)).getInfo()
  end_year = ee.Number(ee.List(year_windows.get(yrwin_i)).get(1)).getInfo()

  eimage = ee.Image(mean_ic_list.get(i))

  task = ee.batch.Export.image.toDrive(
          image=eimage,
          description='tmax_{}_{}_{}'.format(str(start_year), str(end_year), str(mo_i)),
          folder='GEE_Downloads',
          region=area_shp,
          scale=100,
          crs='EPSG:4326',
          maxPixels=1e13)

  task.start()

  if mo_i == 12:
    mo_i = 0
    yrwin_i += 1




In [ ]:
#Future Windows Min Temperature
import ee

ee.Initialize(project='ee-andrewfullhart')

maca_ic = ee.ImageCollection('NASA/NEX-DCP30')

order_months = ee.List([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

year_windows = ee.List([[1974, 2013], [2000, 2029], [2010, 2039], [2020, 2049], [2030, 2059],
                        [2040, 2069], [2050, 2079], [2060, 2089], [2070, 2099]])

area_shp = ee.Geometry.Rectangle([-121, 30, -102, 43], 'EPSG:4326', False)

def year_fn(yrwinobj):
  start_year = ee.Number(ee.List(yrwinobj).get(0))
  end_year = ee.Number(ee.List(yrwinobj).get(1))
  start = ee.Date.fromYMD(ee.Number(start_year), 1, 1)
  end = ee.Date.fromYMD(ee.Number(end_year).add(1), 1, 1)
  year_ic = maca_ic.filterDate(start, end).select('tasmin')
  modelfilter = ee.Filter.Or(
                ee.Filter.eq('scenario', 'historical'),
                ee.Filter.eq('scenario', 'rcp45'))
  year_ic = year_ic.filter(modelfilter)
  year_ic = year_ic.filter(ee.Filter.eq('model', 'CCSM4'))

  def month_fn(mobj):
    m = ee.Number(mobj)
    month_ic = year_ic.filter(ee.Filter.calendarRange(m, m, 'month'))
    mean_im = month_ic.reduce(ee.Reducer.mean()).clip(area_shp).subtract(273.15)
    return mean_im

  mean_ic_list = order_months.map(month_fn)
  return mean_ic_list

nested_mean_ic_list = year_windows.map(year_fn)
mean_ic_list = nested_mean_ic_list.flatten()


yrwin_i = 0
mo_i = 0
for i in range(12*year_windows.size().getInfo()):
  mo_i += 1
  start_year = ee.Number(ee.List(year_windows.get(yrwin_i)).get(0)).getInfo()
  end_year = ee.Number(ee.List(year_windows.get(yrwin_i)).get(1)).getInfo()

  eimage = ee.Image(mean_ic_list.get(i))

  task = ee.batch.Export.image.toDrive(
          image=eimage,
          description='tmin_{}_{}_{}'.format(str(start_year), str(end_year), str(mo_i)),
          folder='GEE_Downloads',
          region=area_shp,
          scale=100,
          crs='EPSG:4326',
          maxPixels=1e13)

  task.start()

  if mo_i == 12:
    mo_i = 0
    yrwin_i += 1


